In [21]:
import json
import csv
import sys
import requests
from pathlib import Path

def allowed_files_only(files: list[str] | None) -> bool:
    if not files:
        return True  # No files — acceptable
    allowed_extensions = {".txt", ".py"}
    return all(Path(f).suffix.lower() in allowed_extensions for f in files)

def download_and_concatenate(files: list[str]) -> str:
    full_content = []
    for url in files:
        try:
            response = requests.get(url)
            response.raise_for_status()
            filename = url.split('/')[-1]
            full_content.append(f"\n\nFile ({filename}) content:\n{response.text}")
        except Exception as e:
            full_content.append(f"\n\nFile ({url}) could not be downloaded: {e}")
    return "".join(full_content)

def export_crypto_txt_py_only(json_path: str | Path, csv_path: str | Path) -> None:
    # --- Load the JSON ---
    with open(json_path, "r", encoding="utf-8") as fp:
        challenges: dict[str, dict] = json.load(fp)

    # --- Columns you want in the CSV ---
    fieldnames = ["name", "description", "difficulty", "id", "flag"]

    # --- Write the filtered rows ---
    with open(csv_path, "w", newline="", encoding="utf-8") as fp:
        writer = csv.DictWriter(fp, fieldnames=fieldnames)
        writer.writeheader()

        for name, meta in challenges.items():
            is_crypto = meta.get("category") == "Cryptography"
            files = meta.get("files")
            description = meta.get("description")
            if is_crypto and allowed_files_only(files) and not "nc jupiter" in description:
                # Base description
                description = meta.get("description", "")
                if files:
                    description += download_and_concatenate(files)
                writer.writerow(
                    {
                        "name":        name,
                        "description": description,
                        "difficulty":  meta.get("difficulty", ""),
                        "id":          meta.get("id", ""),
                        "flag":        meta.get("flag", ""),
                    }
                )

In [22]:
in_json  = "pico_ctf_benchmark.json"
out_csv  = "crypto_no_files.csv"
export_crypto_txt_py_only(in_json, out_csv)
print(f"✓ Wrote {out_csv} with cryptography challenges that have no attached files.")

✓ Wrote crypto_no_files.csv with cryptography challenges that have no attached files.


In [23]:
import pandas as pd

df = pd.read_csv("crypto_no_files.csv")
for i in df["description"]:
    print(i)

Cryptography can be easy, do you know what ROT13 is? cvpbPGS{arkg_gvzr_V'yy_gel_2_ebhaqf_bs_ebg13_Ncualgvd}
You will find the flag after decrypting this file.

File (encrypted.txt) content:
xqkwKBN{z0bib1wv_l3kzgxb3l_555957n3}

The one time pad can be cryptographically secure, but not when you know the key. Can you solve this? We've given you the encrypted flag, key, and a table to help UFJKXQZQUNB with the key of SOLVECRYPTO. Can you use this table to solve it?

File (table.txt) content:
    A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 
   +----------------------------------------------------
A | A B C D E F G H I J K L M N O P Q R S T U V W X Y Z
B | B C D E F G H I J K L M N O P Q R S T U V W X Y Z A
C | C D E F G H I J K L M N O P Q R S T U V W X Y Z A B
D | D E F G H I J K L M N O P Q R S T U V W X Y Z A B C
E | E F G H I J K L M N O P Q R S T U V W X Y Z A B C D
F | F G H I J K L M N O P Q R S T U V W X Y Z A B C D E
G | G H I J K L M N O P Q R S T U V W X Y Z A B C D E F


In [24]:
df

,name,description,difficulty,id,flag
0,Mod 26,"Cryptography can be easy, do you know what ROT...",easy,144,picoCTF{next_time_I'll_try_2_rounds_of_rot13_A...
1,rotation,You will find the flag after decrypting this f...,medium,373,picoCTF{r0tat1on_d3crypt3d_555957f3}
2,Easy1,The one time pad can be cryptographically secu...,medium,43,picoCTF{CRYPTOISFUN}
3,"No Padding, No Problem","Oracles can be your best friend, they will dec...",medium,154,picoCTF{m4yb3_Th0se_m3s54g3s_4r3_difurrent_327...
4,New Caesar,"We found a brand new type of encryption, can y...",medium,158,picoCTF{et_tu?_07d5c0892c1438d2b32600e83dc2b0e5}
5,Dachshund Attacks,What if d is too small? Connect with nc mercur...,medium,159,picoCTF{proving_wiener_8652838}
6,13,"Cryptography can be easy, do you know what ROT...",easy,62,picoCTF{not_too_bad_of_a_problem}
7,Mr-Worldwide,A musician left us a message. What's it mean?\...,medium,40,picoCTF{KODIAK_ALASKA}


In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("crypto_no_files.csv")
df["necessary_info"] = "Flag format is picoCTF{...}"


,Unnamed: 0,name,question,difficulty,id,flag,necessary_info
0,0,Mod 26,"Cryptography can be easy, do you know what ROT...",easy,144,flag{next_time_I'll_try_2_rounds_of_rot13_Aphn...,NaN
1,1,rotation,You will find the flag after decrypting this f...,medium,373,flag{r0tat1on_d3crypt3d_555957f3},NaN
2,2,Easy1,The one time pad can be cryptographically secu...,medium,43,flag{CRYPTOISFUN},NaN
3,3,"No Padding, No Problem","Oracles can be your best friend, they will dec...",medium,154,flag{m4yb3_Th0se_m3s54g3s_4r3_difurrent_3279013},NaN
4,4,New Caesar,"We found a brand new type of encryption, can y...",medium,158,flag{et_tu?_07d5c0892c1438d2b32600e83dc2b0e5},NaN
5,5,Dachshund Attacks,What if d is too small? Connect with nc mercur...,medium,159,flag{proving_wiener_8652838},NaN
6,6,13,"Cryptography can be easy, do you know what ROT...",easy,62,flag{not_too_bad_of_a_problem},NaN
7,7,Mr-Worldwide,A musician left us a message. What's it mean?\...,medium,40,flag{KODIAK_ALASKA},NaN


In [11]:
df["flag"] = df["flag"].str.replace(r"flag\{([^}]*)\}", r"picoCTF{\1}", regex=True)
df

,Unnamed: 0,name,question,difficulty,id,flag,necessary_info
0,0,Mod 26,"Cryptography can be easy, do you know what ROT...",easy,144,picoCTF{next_time_I'll_try_2_rounds_of_rot13_A...,NaN
1,1,rotation,You will find the flag after decrypting this f...,medium,373,picoCTF{r0tat1on_d3crypt3d_555957f3},NaN
2,2,Easy1,The one time pad can be cryptographically secu...,medium,43,picoCTF{CRYPTOISFUN},NaN
3,3,"No Padding, No Problem","Oracles can be your best friend, they will dec...",medium,154,picoCTF{m4yb3_Th0se_m3s54g3s_4r3_difurrent_327...,NaN
4,4,New Caesar,"We found a brand new type of encryption, can y...",medium,158,picoCTF{et_tu?_07d5c0892c1438d2b32600e83dc2b0e5},NaN
5,5,Dachshund Attacks,What if d is too small? Connect with nc mercur...,medium,159,picoCTF{proving_wiener_8652838},NaN
6,6,13,"Cryptography can be easy, do you know what ROT...",easy,62,picoCTF{not_too_bad_of_a_problem},NaN
7,7,Mr-Worldwide,A musician left us a message. What's it mean?\...,medium,40,picoCTF{KODIAK_ALASKA},NaN


In [8]:
df.to_csv("crypto_no_files_flag.csv", index=False)